In [194]:
#各種インストール
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/morikazuhito/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/morikazuhito/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     /Users/morikazuhito/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/morikazuhito/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [195]:
# arxivからデータの取得

import arxiv
import pandas as pd

search_MA = arxiv.Search(query='cat:cs.MA',max_results=100,sort_by=arxiv.SortCriterion.SubmittedDate,sort_order=arxiv.SortOrder.Descending)
search_EM = arxiv.Search(query='cat:econ.EM',max_results=100,sort_by=arxiv.SortCriterion.SubmittedDate,sort_order=arxiv.SortOrder.Descending)
search_AG = arxiv.Search(query='cat:math.AG',max_results=100,sort_by=arxiv.SortCriterion.SubmittedDate,sort_order=arxiv.SortOrder.Descending)
search_CB = arxiv.Search(query='cat:q-bio.CB',max_results=100,sort_by=arxiv.SortCriterion.SubmittedDate,sort_order=arxiv.SortOrder.Descending)
search_ML = arxiv.Search(query='cat:stat.ML',max_results=100,sort_by=arxiv.SortCriterion.SubmittedDate,sort_order=arxiv.SortOrder.Descending)
search_list = [search_MA,search_EM,search_AG,search_CB,search_ML]

sum_list = []
ti_list = []
cat_list = []

for i ,search in enumerate(search_list):
    for result in search.results():
        sum_list.append(result.summary)
        ti_list.append(result.title)
        cat_list.append(i)

In [196]:
"""# お試し: 一つのカテゴリのデータを使用
import arxiv
import pandas as pd

search = arxiv.Search(query='cat:cs.MA',max_results=100,sort_by=arxiv.SortCriterion.SubmittedDate,sort_order=arxiv.SortOrder.Descending)

sum_list = []
ti_list = []
cat_list = []


for result in search.results():
    sum_list.append(result.summary)
    ti_list.append(result.title)
    cat_list.append(result.categories)"""


"# お試し: 一つのカテゴリのデータを使用\nimport arxiv\nimport pandas as pd\n\nsearch = arxiv.Search(query='cat:cs.MA',max_results=100,sort_by=arxiv.SortCriterion.SubmittedDate,sort_order=arxiv.SortOrder.Descending)\n\nsum_list = []\nti_list = []\ncat_list = []\n\n\nfor result in search.results():\n    sum_list.append(result.summary)\n    ti_list.append(result.title)\n    cat_list.append(result.categories)"

In [197]:
# 単語列のリストに分割
docs = [summary.split() for summary in sum_list]
print(docs[:5])
print("num of docs:", len(docs))

[['Smooth', 'coordination', 'within', 'a', 'swarm', 'robotic', 'system', 'is', 'essential', 'for', 'the', 'effective', 'execution', 'of', 'collective', 'robot', 'missions.', 'Having', 'efficient', 'communication', 'is', 'key', 'to', 'the', 'successful', 'coordination', 'of', 'swarm', 'robots.', 'This', 'paper', 'proposes', 'a', 'new', 'communication-efficient', 'decentralized', 'cooperative', 'reinforcement', 'learning', 'algorithm', 'for', 'coordinating', 'swarm', 'robots.', 'It', 'is', 'made', 'efficient', 'by', 'hierarchically', 'building', 'on', 'the', 'use', 'of', 'local', 'information', 'exchanges.', 'We', 'consider', 'a', 'case', 'study', 'application', 'of', 'maze', 'solving', 'through', 'cooperation', 'among', 'a', 'group', 'of', 'robots,', 'where', 'the', 'time', 'and', 'costs', 'are', 'minimized', 'while', 'avoiding', 'inter-robot', 'collisions', 'and', 'path', 'overlaps', 'during', 'exploration.', 'With', 'a', 'solid', 'theoretical', 'basis,', 'we', 'extensively', 'analyze'

## 前処理

In [198]:
# nltkのストップワードリスト
en_stop = nltk.corpus.stopwords.words('english')
en_stop= ["``", "/", ";", "--", ":", ")", "(", '"', '&', "'", '-', "?", ">", "<"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
        +["propose","study","data","method","result","show","use","using","model","$","also","one","two","three"]          \
         +en_stop

In [199]:
#前処理関数の作成

from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def preprocess_word(word, stopwordset):
    
    #1.make words lower  example: Python =>python
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopword  example: the => (None) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [200]:
#前処理の出力確認
#before
print(docs[0][:25]) 

#after
print(preprocess_documents(docs)[0][:25])

['Smooth', 'coordination', 'within', 'a', 'swarm', 'robotic', 'system', 'is', 'essential', 'for', 'the', 'effective', 'execution', 'of', 'collective', 'robot', 'missions.', 'Having', 'efficient', 'communication', 'is', 'key', 'to', 'the', 'successful']
['smooth', 'coordination', 'within', 'swarm', 'robotic', 'system', 'essential', 'effective', 'execution', 'collective', 'robot', 'missions.', 'efficient', 'communication', 'key', 'successful', 'coordination', 'swarm', 'robots.', 'paper', 'new', 'communication-efficient', 'decentralize', 'cooperative', 'reinforcement']


## LDA

In [201]:
import gensim
from gensim import corpora

In [202]:
#documentを，gensim LDAが読み込めるデータ構造にする

#辞書の作成
dictionary = corpora.Dictionary(preprocess_documents(docs))
#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

In [203]:
print(dictionary.token2id)

{'accuracy': 0, 'algorithm': 1, 'among': 2, 'analyze': 3, 'application': 4, 'avoid': 5, 'basis,': 6, 'building': 7, 'case': 8, 'collective': 9, 'collision': 10, 'communication': 11, 'communication-degraded': 12, 'communication-efficient': 13, 'consider': 14, 'cooperation': 15, 'cooperative': 16, 'coordinate': 17, 'coordination': 18, 'core': 19, 'costs': 20, 'coverage': 21, 'decentralize': 22, 'demonstrate': 23, 'effective': 24, 'efficiency': 25, 'efficient': 26, 'environments.': 27, 'essential': 28, 'evaluate': 29, 'even': 30, 'exchanges.': 31, 'execution': 32, 'exploration.': 33, 'extensively': 34, 'group': 35, 'hierarchically': 36, 'high': 37, 'higher': 38, 'information': 39, 'inter-robot': 40, 'key': 41, 'learning': 42, 'local': 43, 'loss': 44, 'low': 45, 'make': 46, 'maze': 47, 'minimize': 48, 'missions.': 49, 'network': 50, 'new': 51, 'overlap': 52, 'packet': 53, 'paper': 54, 'path': 55, 'percentage': 56, 'range': 57, 'realistic': 58, 'reducing': 59, 'reinforcement': 60, 'robot': 

In [204]:
print(corpus_[-1][:10]) #文章での出現順でなく辞書IDの若い順なことに注意

[(1, 3), (14, 1), (42, 3), (51, 1), (57, 1), (161, 1), (171, 1), (176, 1), (205, 1), (280, 1)]


## LDA学習

In [205]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=5,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

In [206]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.008*"cell" + 0.004*"system" + 0.004*"learning" + 0.004*"algorithm" + 0.003*"provide" + 0.003*"prove" + 0.003*"policy" + 0.003*"different" + 0.003*"estimate" + 0.003*"base" + 0.003*"parameter" + 0.002*"however," + 0.002*"network" + 0.002*"agent" + 0.002*"function"')
(1, '0.009*"cell" + 0.004*"algorithm" + 0.003*"approach" + 0.003*"new" + 0.003*"provide" + 0.003*"introduce" + 0.003*"base" + 0.003*"optimal" + 0.002*"process" + 0.002*"problem" + 0.002*"performance" + 0.002*"learning" + 0.002*"demonstrate" + 0.002*"analysis" + 0.002*"first"')
(2, '0.004*"learning" + 0.003*"agent" + 0.003*"problem" + 0.003*"algorithm" + 0.003*"base" + 0.003*"system" + 0.003*"network" + 0.003*"space" + 0.002*"provide" + 0.002*"prove" + 0.002*"estimate" + 0.002*"approach" + 0.002*"consider" + 0.002*"different" + 0.002*"function"')
(3, '0.007*"cell" + 0.004*"agent" + 0.003*"develop" + 0.003*"approach" + 0.003*"algorithm" + 0.003*"learning" + 0.003*"system" + 0.003*"network" + 0.002*"new" + 0.002*"differe

In [207]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(0, 0.29142287), (4, 0.70532554)]
document ID 1:[(4, 0.9970969)]
document ID 2:[(4, 0.99520546)]
document ID 3:[(4, 0.99653363)]
document ID 4:[(0, 0.9960503)]
document ID 5:[(2, 0.99495685)]
document ID 6:[(2, 0.99502254)]
document ID 7:[(3, 0.99562466)]
document ID 8:[(0, 0.99743956)]
document ID 9:[(0, 0.9962735)]


In [208]:
# 各文章のトピックを取得
topic_list = [ldamodel.get_document_topics(document)[0][0] for document in corpus_]
print(type(topic_list))
print(topic_list)

<class 'list'>
[0, 4, 4, 4, 0, 2, 2, 3, 0, 0, 4, 1, 0, 0, 2, 4, 2, 1, 1, 1, 3, 2, 2, 1, 0, 0, 2, 4, 0, 0, 1, 1, 0, 0, 0, 4, 3, 3, 2, 0, 3, 1, 4, 2, 1, 4, 4, 0, 1, 1, 2, 1, 2, 2, 4, 2, 4, 1, 0, 4, 4, 4, 4, 1, 2, 3, 2, 4, 0, 2, 2, 4, 2, 3, 0, 0, 2, 2, 1, 1, 4, 1, 0, 0, 0, 4, 4, 0, 3, 2, 4, 0, 0, 4, 4, 1, 1, 3, 0, 4, 1, 1, 4, 0, 1, 0, 4, 1, 0, 4, 1, 0, 1, 3, 4, 0, 1, 1, 4, 1, 1, 1, 4, 3, 4, 1, 0, 4, 1, 1, 2, 0, 1, 1, 0, 2, 3, 2, 2, 0, 4, 1, 2, 3, 1, 1, 2, 0, 1, 2, 4, 1, 0, 1, 1, 0, 0, 4, 3, 4, 4, 4, 0, 0, 3, 4, 4, 4, 1, 0, 4, 1, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 1, 4, 4, 0, 1, 2, 4, 1, 4, 0, 0, 4, 0, 4, 2, 4, 1, 3, 2, 4, 2, 2, 3, 2, 2, 0, 3, 0, 0, 0, 4, 4, 0, 0, 2, 4, 0, 3, 0, 0, 0, 2, 1, 0, 0, 0, 3, 1, 2, 1, 4, 0, 0, 2, 0, 3, 2, 0, 3, 0, 0, 0, 4, 0, 1, 0, 2, 4, 0, 0, 0, 1, 2, 2, 1, 0, 4, 0, 0, 0, 2, 0, 0, 3, 3, 0, 2, 0, 0, 2, 4, 4, 2, 0, 0, 0, 3, 4, 2, 4, 0, 0, 0, 4, 1, 0, 0, 4, 0, 2, 1, 1, 4, 3, 2, 0, 2, 0, 4, 1, 0, 0, 4, 1, 3, 1, 4, 4, 3, 1, 1, 1, 3, 1, 1, 1, 0, 3, 0, 2, 1, 0, 0, 1, 1, 1, 

In [209]:
# カテゴリとトピックの対応を混合行列により調べる
import numpy as np
# カテゴリとトピックのクラス数を取得
classes_cat = np.unique(cat_list)
classes_top = np.unique(topic_list)
confusion_matrix = np.zeros((len(classes_cat), len(classes_top)), dtype=int)
for a, b in zip(cat_list, topic_list):
    i = np.where(classes_cat == a)[0][0]
    j = np.where(classes_top == b)[0][0]
    confusion_matrix[i, j] += 1
# 混合行列を表示する
print("Confusion Matrix:")
print(confusion_matrix)

Confusion Matrix:
[[26 19 21  9 25]
 [27 31 10  7 25]
 [45  9 21 10 15]
 [28 37  7 15 13]
 [29 24  6 15 26]]


In [210]:
categories = cat_list

In [211]:
# トピック分布の出力
for n in range(0,500,50):  
    #n番目のdocumentのcategory
    print("カテゴリ: ",end="")
    print(categories[n])
    
    #n番目のdocumentのトピック分布
    print("トピック分布: ",end="")
    print(ldamodel.get_document_topics(corpus_[n]))


    #n番目のdocumentのタイトル
    print("タイトル: ",end="")
    print(ti_list[n])
    
    # 要約の原文
    # print(" ".join(docs[n]))
    
    print()

カテゴリ: 0
トピック分布: [(0, 0.29065838), (4, 0.70609003)]
タイトル: Communication-Efficient Reinforcement Learning in Swarm Robotic Networks for Maze Exploration

カテゴリ: 0
トピック分布: [(2, 0.99566925)]
タイトル: Q-SHED: Distributed Optimization at the Edge via Hessian Eigenvectors Quantization

カテゴリ: 1
トピック分布: [(1, 0.9958086)]
タイトル: A Policy Gradient Method for Confounded POMDPs

カテゴリ: 1
トピック分布: [(4, 0.99531555)]
タイトル: Adjustment with Many Regressors Under Covariate-Adaptive Randomizations

カテゴリ: 2
トピック分布: [(2, 0.9890309)]
タイトル: Extension Groups of Tautological Bundles on Punctual Quot Schemes of Curves

カテゴリ: 2
トピック分布: [(0, 0.9898661)]
タイトル: Regularized Integrals on Configuration Spaces of Riemann Surfaces and Cohomological Pairings

カテゴリ: 3
トピック分布: [(4, 0.9964698)]
タイトル: A model of replicating coupled oscillators generates naturally occurring cell networks

カテゴリ: 3
トピック分布: [(0, 0.30217198), (1, 0.17984962), (2, 0.51666135)]
タイトル: There's Plenty of Room Right Here: Biological Systems as Evolved, Overload

## 可視化

In [212]:
#import pyLDAvis.gensim
import pyLDAvis.gensim_models
# jupyter notebook で結果を表示するのに必要
pyLDAvis.enable_notebook()

In [213]:
#全documentを学習に用いた場合結構時間がかかる(20min~)
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)
